In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals
try:
  # Colab only
  %tensorflow_version 2.x
except Exception:
  pass

from __future__ import absolute_import, division, print_function, unicode_literals
import tensorflow as tf
from tensorflow import keras
from matplotlib import pyplot as plt
import numpy as np

In [8]:
layer = tf.keras.layers
print("tensorflow version check : ", tf.__version__)
print("gpu check : ", tf.test.is_gpu_available())

tensorflow version check :  2.0.0-rc2
gpu check :  True


# 1. get dataset on memory

In [0]:
cifar100 = tf.keras.datasets.cifar100
train, test = cifar100.load_data(label_mode='coarse')
train_ds = tf.data.Dataset.from_tensor_slices(
    (train[0], train[1])).shuffle(5000).batch(64)
test_ds = tf.data.Dataset.from_tensor_slices(
    (test[0], test[1])).shuffle(5000).batch(64)

In [0]:
IMG_SIZE = 128
LEARNING_RATE = 0.0001
fine_tune_at = 170

# 2. Add additional processing into your dataset

In [0]:
def parse_image(image, label):
    image = tf.image.resize(image, [IMG_SIZE, IMG_SIZE])
    return image, label


train_ds = train_ds.map(parse_image)
test_ds = test_ds.map(parse_image)
IMG_SHAPE = (IMG_SIZE, IMG_SIZE, 3)

# 3. Create the base model from the pre-trained convnets

In [13]:
# Create the base model from the pre-trained model MobileNet V2
base_model = tf.keras.applications.ResNet50(input_shape=IMG_SHAPE,
                                            include_top=False,
                                            weights='imagenet')

94773248/94765736 [==============================] - 1s 0us/step


### 3.1 Freeze the convolutional base and use it as feature extractor

In [16]:
# Let's take a look to see how many layers are in the base model
print("Number of layers in the base model: ", len(base_model.layers))
base_model.trainable = False
# 학습이 안되게 얼려버렸다.

Number of layers in the base model:  175


In [0]:
base_model.summary()

### 3.2 finetune the part of base model

In [15]:
base_model.trainable = True
# Let's take a look to see how many layers are in the base model
print("Number of layers in the base model: ", len(base_model.layers))
# Fine tune from this layer onwards
# Freeze all the layers before the `fine_tune_at` layer

for l in base_model.layers[:fine_tune_at]:
    print(l.trainable)
    l.trainable = False
    print('now trainable is false')

Number of layers in the base model:  175
True
now trainable is false
True
now trainable is false
True
now trainable is false
True
now trainable is false
True
now trainable is false
True
now trainable is false
True
now trainable is false
True
now trainable is false
True
now trainable is false
True
now trainable is false
True
now trainable is false
True
now trainable is false
True
now trainable is false
True
now trainable is false
True
now trainable is false
True
now trainable is false
True
now trainable is false
True
now trainable is false
True
now trainable is false
True
now trainable is false
True
now trainable is false
True
now trainable is false
True
now trainable is false
True
now trainable is false
True
now trainable is false
True
now trainable is false
True
now trainable is false
True
now trainable is false
True
now trainable is false
True
now trainable is false
True
now trainable is false
True
now trainable is false
True
now trainable is false
True
now trainable is false
True
no

# 4. model redefinition and training

In [0]:
inputs = keras.Input(shape=(128, 128, 3))
feature_batch = base_model(inputs)
global_average_layer = layer.Flatten()(feature_batch)
prediction = layer.Dense(100, activation='softmax')(global_average_layer)
model = keras.Model(inputs, prediction)

In [19]:
model.summary()

model.compile(optimizer=tf.keras.optimizers.Adam(LEARNING_RATE),
              loss=tf.keras.losses.sparse_categorical_crossentropy,
              metrics=[tf.keras.metrics.sparse_categorical_accuracy])

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 128, 128, 3)]     0         
_________________________________________________________________
resnet50 (Model)             (None, 4, 4, 2048)        23587712  
_________________________________________________________________
flatten (Flatten)            (None, 32768)             0         
_________________________________________________________________
dense (Dense)                (None, 100)               3276900   
Total params: 26,864,612
Trainable params: 3,276,900
Non-trainable params: 23,587,712
_________________________________________________________________


In [20]:
print(len(model.trainable_variables))

print('========================Training===============================')

model.fit(train_ds, epochs=5, validation_data=test_ds)

# print('========================Evaluation===============================')
#
# test_loss, test_acc = model.evaluate(test_ds)

del model
keras.backend.clear_session()

2
========================Training===============================
Epoch 1/5
782/782 [==============================] - 173s 221ms/step - loss: 1.5625 - sparse_categorical_accuracy: 0.5819 - val_loss: 0.0000e+00 - val_sparse_categorical_accuracy: 0.0000e+00
Epoch 2/5
782/782 [==============================] - 164s 210ms/step - loss: 0.7422 - sparse_categorical_accuracy: 0.7719 - val_loss: 2.7673 - val_sparse_categorical_accuracy: 0.4796
Epoch 3/5
782/782 [==============================] - 164s 210ms/step - loss: 0.4488 - sparse_categorical_accuracy: 0.8538 - val_loss: 2.7404 - val_sparse_categorical_accuracy: 0.4911
Epoch 4/5
782/782 [==============================] - 165s 212ms/step - loss: 0.2993 - sparse_categorical_accuracy: 0.9035 - val_loss: 2.8775 - val_sparse_categorical_accuracy: 0.4919
Epoch 5/5
782/782 [==============================] - 166s 212ms/step - loss: 0.2092 - sparse_categorical_accuracy: 0.9332 - val_loss: 2.9266 - val_sparse_categorical_accuracy: 0.4946


https://benchmarks.ai/cifar-100

In [0]:
del model
keras.backend.clear_session()